In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         # print(os.path.join(dirname, filename))
#         pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

In [ ]:
!pip install ../input/required-wheels/base58-2.1.0-py3-none-any.whl
!pip install ../input/required-wheels/astor-0.8.1-py2.py3-none-any.whl
!pip install ../input/required-wheels/validators-0.18.2-py3-none-any.whl
!pip install ../input/required-wheels/pydeck-0.6.2-py2.py3-none-any.whl
!pip install ../input/required-wheels/watchdog-2.1.1-py3-none-manylinux2014_x86_64.whl

!pip install ../input/required-wheels/streamlit-0.82.0-py2.py3-none-any.whl

!pip install ../input/required-wheels/ipykernel-5.5.5-py3-none-any.whl
!pip install ../input/required-wheels/fsspec-0.9.0-py3-none-any.whl

In [ ]:
! pip install ../input/required-wheels/simpletransformers-0.61.4-py3-none-any.whl

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
import logging
import pickle
import json
import re
import time
import math

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample_submission_df = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_split_size = math.floor(2834*0.8)

In [ ]:
train_df_80 = train_df[:train_split_size]

In [ ]:
train_df_20 = train_df[train_split_size:]

In [ ]:
train_df_80.shape, train_df_20.shape

In [ ]:
train_df_20.head()

In [ ]:
train_df_20[['excerpt', 'target']]

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.learning_rate = 5e-6
model_args.regression = True
model_args.overwrite_output_dir = True

# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "../input/huggingface-roberta-variants/roberta-base/roberta-base",
    num_labels=1,
    use_cuda=True,
    args=model_args
)

# Train the model
model.train_model(train_df_80[['excerpt', 'target']])

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(train_df_20[['excerpt', 'target']])

In [ ]:
# Make predictions with the model
predictions = []
for row in test_df.itertuples():
    prediction, _ = model.predict([row.excerpt])
    predictions.append(float(prediction))

In [ ]:
predictions

In [ ]:
sample_submission_df['target'] = pd.Series(predictions)

In [ ]:
sample_submission_df

In [ ]:
sample_submission_df.to_csv('submission.csv', index=False)